# Save rows from HUMMUS and open food facts  

In [ ]:
from file_creation import extract_rows

rows = 3

input_file = "../csv_file/pp_recipes.csv"
output_file = "../csv_file/pp_recipes_rows.csv"
extract_rows(input_file, output_file, rows=rows)

input_file = "../csv_file/pp_members.csv"
output_file = "../csv_file/pp_members_rows.csv"
extract_rows(input_file, output_file, rows=rows)

input_file = "../csv_file/pp_reviews.csv"
output_file = "../csv_file/pp_reviews_rows.csv"
extract_rows(input_file, output_file, rows=rows)

input_file = "../csv_file/off.csv"
output_file = "../csv_file/off_rows.csv"
extract_rows(input_file, output_file, rows=rows, delimiter="\t")

# Save casual row from off to create a test dataset 

In [ ]:
from file_creation import extract_rows_random

rows = 200
input_file = "../csv_file/off.csv"
output_file = "../csv_file/off_rows_casual.csv"
extract_rows_random(input_file, output_file, rows=rows, delimiter="\t")

# Save recipe name from hummus and open food facts

In [ ]:
from file_creation import extract_recipes

rows = 3

input_file = "../csv_file/pp_recipes.csv"
output_file = "../csv_file/hummus_recipes.csv"
extract_recipes(input_file, output_file, column_name="title", rows=rows)

input_file = "../csv_file/off.csv"
output_file = "../csv_file/off_recipes.csv"
extract_recipes(
    input_file,
    output_file,
    column_name="product_name",
    rows=rows,
    delimiter1="\t",
    delimiter2=" ",
)

# Save ingredients name from foodKg

In [ ]:
from file_creation import extract_ingredients_foodkg

input_file = "../csv_file/pp_recipes.csv"
output_file = "../csv_file/ingredients_food_kg.csv"
extract_ingredients_foodkg(
    input_file=input_file, output_file=output_file, min_occurrences=0
)

# Save all adjectives from FoodKG food

In [ ]:
from file_creation import extract_adjectives_foodkg

input_file = "../csv_file/ingredients_food_kg.csv"
output_file = "../csv_file/aggettivi_FoodKg.csv"
extract_adjectives_foodkg(
    input_file=input_file, output_file=output_file, text_column="ingredient"
)

# Save all tags from HUMMUS

In [ ]:
from file_creation import extract_tags_unique

extract_tags_unique(
    "../csv_file/pp_recipes.csv", "../csv_file/tag_hummus_frequency.csv"
)

# Save brand that appear more than n times


In [ ]:
from file_creation import extract_clean_brands

input_file = "../csv_file/off.csv"
output_file = "../csv_file/brands.csv"
extract_clean_brands(input_file=input_file, output_file=output_file, n=5)

# Brand filtering with the food expert model

clean the brand file from food names and save it as a new file

In [ ]:
from file_creation import brand_filtering

file = "../csv_file/brands.csv"
output_file = "../csv_file/brands_filtered.csv"
brand_filtering(file, output_file)

# Create a file with some member description for test the attribute extractor model

In [ ]:
from file_creation import extract_description

rows = 20
input_file = "../csv_file/pp_members.csv"
output_file = "../csv_file/user_description.csv"
extract_description(input_file, output_file, column_name="member_description", rows=rows)

# Add the group tag to the recipes file

(In future work)

In [ ]:
#from file_creation import transform_groups_in_tags
#
#transform_groups_in_tags(
#    "../csv_file/recipes_pp_binned.csv",
#    "../csv_file/pp_recipes_groups.csv",
#    ["protein_group","calories_group","calories_from_fat_group","total_fat_group","saturated_fat_group","cholesterol_group","sodium_group","carbohydrate_group","dietary_fiber_group","sugars_group","duration_group"],
#    delimiter1=",",
#    delimiter2=";"
#)

save all the member description with their associated member id


In [ ]:
from file_creation import save_some_column

input_csv = "../csv_file/pp_members.csv"
output_csv = "../csv_file/member_description_list.csv"
column1 = "member_id"
column2 = "member_description"

save_some_column(input_csv, output_csv, column1, column2)

In [ ]:
from file_creation import save_some_column

input_csv = "../csv_file/pp_reviews.csv"
output_csv = "../csv_file/member_review_list.csv"
column1 = "member_id"
column2 = "text"

save_some_column(input_csv, output_csv, column1, column2)

In [ ]:
import pandas as pd

file1 = "../csv_file/member_description_list.csv"
file2 = "../csv_file/member_review_list.csv"
output_csv = "../csv_file/member_complete_info_list.csv"

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

merged_df = df1.merge(df2, on="member_id", how="left")

def build_information(row):
    info = [f"member description: {row['member_description']}"]
    reviews = row['text'].dropna().tolist() if isinstance(row['text'], pd.Series) else [row['text']]
    for i, review in enumerate(reviews, start=1):
        info.append(f"review number {i}: {review}")
    return "; ".join(info)

merged_df["information"] = merged_df.apply(build_information, axis=1)

final_df = merged_df[["member_id", "information"]]

final_df.to_csv(output_csv, index=False)
print(f"File saved: {output_csv}")


This script is used to handle a problem encountered in the encoding process

In [28]:
import pandas as pd
import os
import csv

def change_csv_delimiter(file_path, current_delimiter_char, new_delimiter_char, max_rows_to_read=None):
    df = pd.read_csv(
        file_path,
        sep=current_delimiter_char,
        nrows=max_rows_to_read,
        engine='python',
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL,
        doublequote=True,
        on_bad_lines='skip',
        dtype=str
    )

    df.replace(r'^\s*$', pd.NA, regex=True, inplace=True)
    df.dropna(how='all', inplace=True)

    dir_name = os.path.dirname(file_path)
    base_name = os.path.basename(file_path)
    temp_base_name = ".temp_conversion_" + base_name
    temp_output_file_path = os.path.join(dir_name, temp_base_name)
    
    df.to_csv(temp_output_file_path, sep=new_delimiter_char, index=False, quoting=csv.QUOTE_MINIMAL)
    
    os.remove(file_path)
    os.rename(temp_output_file_path, file_path)

path_to_csv_file_for_modification = "../csv_file/pp_recipes.csv"
original_file_delimiter_char = ","
target_file_delimiter_char = ";"
row_processing_read_limit = None

change_csv_delimiter(
    file_path=path_to_csv_file_for_modification,
    current_delimiter_char=original_file_delimiter_char,
    new_delimiter_char=target_file_delimiter_char,
    max_rows_to_read=row_processing_read_limit
)